In [6]:
from stat import S_ISREG, ST_CTIME, ST_MODE
import os, sys, time, re, shutil, json
from flask import Flask, render_template
dirpath = '/mnt/sda4/Project/FinalProject/flask/dataset'
os.chdir(dirpath)
dataset = []
listfil = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for file in listfil:
    z = re.match("(.+.pcap)",file)
    if z:
        dataset.append(file)
print(dataset)
# Sebelum data di proses, data harus di download terlebih dahulu
# Setelah data di download, langsung balik ke menu awal (halaman awal)
# Setelah balik ke halaman awal, ada halaman hasil
# Ditunggu sampai hasilnya di dapatkan
# Caranya adalah data yang sudah di download akan di proses oleh data concurrent prosesor
# Untuk setiap datanya akan di proses yang paling lama diproses paling pertama
# Yang paling baru diproses paling terakhir
print("Data yang terbaru adalah", dataset[len(dataset)-1])
print("Data yang terlama adalah", dataset[0])
# Keluarkan status setelah data di proses (dilist)

def proses(data):
    # Fungsi ini harus dijalankan pertama kali setelah data di upload
    # Buat directory dengan nama file yang dihilangkan pcapnya
    dirname = re.sub(r'.pcap','',data)
    if(os.path.exists(dirname)):
        print('directory sudah ada')
    else:
        print('Membuat file status')
        with open(dirname+'.status', 'w') as outfile:
            print('Inisialisasi status 0: belum di analisis...')
            outfile.write('0')
        print('directory tidak ada')
        print('membuat directory..')
        os.mkdir(dirname)
        print(dirname+" Telah di buat")
        print('membuat directory CNN..')
        os.mkdir(dirname+'/CNN')
        print(dirname+'/CNN'+" Telah di buat")
        print('membuat directory LSTM..')
        os.mkdir(dirname+'/LSTM')
        print(dirname+'/LSTM'+" Telah di buat")

def hapus(data):
    # Hapus data pcap
    print('Menghapus '+data+'...')
    try:
        os.remove(data)
        print(data+' terhapus!')
    except Exception as e: 
        print(e)
    # Hapus directory
    dirname = re.sub(r'.pcap','',data)
    print('Menghapus '+dirname+'...')
    try:
        shutil.rmtree(dirname)
        print(dirname+' terhapus!')
    except Exception as e:
        print(e)
    # Hapus status
    try:
        os.remove(dirname+'.status')
        print(dirname+'.status terhapus')
    except Exception as e:
        print(e)

def hilang(data, daftar):
    # Mxehgapus semua yang berkaitan dengan data
    hapus(data)
    # Menghapus data dari list daftar
    daftar.remove(data)
    # Mengembalikan nilai daftar
    return daftar

def resettest(dataset):
    for i in dataset:
        a = re.sub(r'.pcap','',i)
        print("Menghapus direktori "+a+"...")
        shutil.rmtree(a)
        print("Direktori "+a+" sudah terhapus")
        print("Mereset status menjadi 0 kembali...")
        with open(a+'.status','w') as tulis:
            tulis.write('0')
        print("Status sudah direset menjadi 0")
    
def bacastatus(data):
    statusname = re.sub(r'.pcap','.status',data)
    with open(statusname) as bacaan:
        status = bacaan.read()
    if (status == '2'):
        print("Sudah di analisis")
    elif (status == '1'):
        print("Sedang di analisis")
    elif (status == '0'):
        print("Belum di analisis")
    else:
        print("Status "+status+": Tidak diketahui")

# Menulis dataset setiap kali upload data pcap
with open('dataset.json', 'w') as outfile:
    json.dump(dataset, outfile)

# Membaca dataset setiap kali upload data pcap ini dibaca oleh main service untuk mengecek apakah
# Data sudah ada atau tidak
# Data pcap akan diantrikan di program ini untuk mendapatkan hasilnya
# Setelah antrian selesai maka hasil pemrosesan data dapat langsung diperoleh di halaman yang dapat diakses
# Jika status menandakan sudah selsai hasil pemrosesannya
with open('dataset.json') as json_file:
    dataout = json.load(json_file)

['300.pcap', '400.pcap', '500.pcap', '600.pcap', '700.pcap', '800.pcap', '900.pcap', 'rbot.pcap', 'neris.pcap', 'snort.log.1576438660.pcap', 'snort.log.1576438833.pcap', 'svchosta.pcap']
Data yang terbaru adalah svchosta.pcap
Data yang terlama adalah 300.pcap


In [10]:
hilang(dataset[len(dataset)-2], dataset)

Menghapus snort.log.1576438833.pcap...
snort.log.1576438833.pcap terhapus!
Menghapus snort.log.1576438833...
snort.log.1576438833 terhapus!
snort.log.1576438833.status terhapus


['300.pcap',
 '400.pcap',
 '500.pcap',
 '600.pcap',
 '700.pcap',
 '800.pcap',
 '900.pcap',
 'rbot.pcap',
 'neris.pcap',
 'snort.log.1576438660.pcap',
 'svchosta.pcap']

In [146]:
os.listdir('dataset')

dataset = []
os.chdir('./dataset')
listfil = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for file in listfil:
    z = re.match("(.+.pcap)",file)
    if z:
        dataset.append(file)
print(dataset)
os.chdir(owd)

['300.pcap', '400.pcap', '500.pcap', '600.pcap', '700.pcap', '800.pcap', '900.pcap', 'rbot.pcap', 'neris.pcap', 'snort.log.1576438660.pcap', 'snort.log.1576438833.pcap']


In [82]:
for file in listfil:
    z = re.match("(\w+.pcap)",file)
    if z:
        print(file)

300.pcap
400.pcap
500.pcap
600.pcap
700.pcap
800.pcap
900.pcap
abc.pcap
123.pcap


In [68]:
from flask import Flask, jsonify
app = Flask(__name__)
@app.route('/summary')
def summary():
    return jsonify(dataset)
if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Dec/2019 21:26:51] "GET /summary HTTP/1.1" 200 -


In [109]:
json.loads(b'[["300.pcap","2"],["400.pcap","2"],["500.pcap","2"],["600.pcap","2"],["700.pcap","2"],["800.pcap","2"],["900.pcap","2"],["rbot.pcap","2"],["neris.pcap","2"]]')[0][0]

'300.pcap'

In [126]:
owd = os.getcwd()
listsnort = os.listdir('test')
for i in listsnort:
    a = re.sub(r'$','.pcap',i)
    print("Menyalin file dari test ke dataset...")
    shutil.copyfile("test/"+i,"dataset/"+a)
    print("test/"+i+" telah disalin ke dataset/"+a)
os.listdir('dataset')


Menyalin file dari test ke dataset...
test/snort.log.1576438660 telah disalin ke dataset/snort.log.1576438660.pcap
Menyalin file dari test ke dataset...
test/snort.log.1576438833 telah disalin ke dataset/snort.log.1576438833.pcap


['.executor.py.swp',
 '300',
 '300.pcap',
 '300.status',
 '400',
 '400.pcap',
 '400.status',
 '500',
 '500.pcap',
 '500.status',
 '600',
 '600.pcap',
 '600.status',
 '700',
 '700.pcap',
 '700.status',
 '800',
 '800.pcap',
 '800.status',
 '900',
 '900.pcap',
 '900.status',
 'dataset.json',
 'executor.py',
 'neris',
 'neris.pcap',
 'neris.status',
 'rbot',
 'rbot.pcap',
 'rbot.status',
 'snort.log.1576438660.pcap',
 'snort.log.1576438833.pcap']